In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

In [ ]:
import ollama


def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers

    Args:
      a: The first integer number
      b: The second integer number

    Returns:
      int: The sum of the two numbers
    """
    return a + b


response = ollama.chat(
    "llama3.1",
    messages=[{"role": "user", "content": "What is 10 + 10?"}],
    tools=[add_two_numbers],  # Actual function reference
)


if response.message.tool_calls:
    name = response.message.tool_calls[0].tool_name
    print(f"🦙 The tool name is {name}")
    args = response.message.tool_calls[0].args

    if args:
        print(f"🦙 The arguments are {args}")


print(response, response.message.tool_calls)

In [ ]:
print(f"Listing available models...")

pretty_formatted_list = list(ollama.list())
models = pretty_formatted_list[0][0]
print(pretty_formatted_list)

In [45]:
from typing import Union
from pydantic import BaseModel
import json
import traceback

DEFAULT_SYSTEM_PROMPT = (
    "You are an intelligent assistant. You are helping the user with their query."
)
DEFAULT_TEMPERATURE = 0.5
DEFAULT_MAX_TOKENS = 100
DEFAULT_OLLAMA_MODEL = "phi4"
DEFAULT_VERBOSE = True
DEFAULT_DEBUG = True


def build_dummy_pydantic_object(schema: BaseModel) -> BaseModel:
    """
    Build a dummy Pydantic object using the given schema.

    Args:
      schema: The Pydantic schema to build the object from

    Returns:
      BaseModel: The dummy Pydantic object
    """
    return schema()


def generate_object(
    prompt: str,
    response_model: BaseModel,
    system=DEFAULT_SYSTEM_PROMPT,
    model=DEFAULT_OLLAMA_MODEL,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    debug=DEFAULT_DEBUG,
    verbose=DEFAULT_VERBOSE,
) -> Union[BaseModel, None]:
    """Generates an object using the OpenAI API and given response model."""
    try:
        if verbose or debug:
            print(f"Generating object for prompt: {prompt}")

        prompt_with_structured_output = f"""
            Prompt: {prompt} 
            SCHEMA: {build_dummy_pydantic_object(response_model).model_dump_json()}
            RESPOND IN JSON FORMAT
        """

        if debug:
            params = {
                "prompt": prompt_with_structured_output,
                "system": system,
                "temperature": temperature,
                "max_tokens": max_tokens,
                "model": model,
            }
            params = json.dumps(params, indent=2)
            print(f"Params: {params}")

        response = ollama.chat(
            model=model,
            messages=[
                # {"role": "system", "content": system},
                {"role": "user", "content": prompt},
            ],
            format="json",
        )

        response_json = response.message.content  # Get the response content
        print(response_json)
        response_obj = json.loads(response_json)
        response_structured = response_model.model_validate(response_obj)

        if verbose or debug:
            print("Object generated successfully. 🎉")

        if debug:
            print(f"EasyLLM Response: {response_json}")
        return response_structured
    except Exception as e:
        print(f"Failed to generate object. Error: {str(e)}")
        if debug:
            traceback.print_exc()
        return None

In [ ]:
from pydantic import BaseModel


class Joke(BaseModel):
    joke: str = (
        "Why did the scarecrow win an award? Because he was outstanding in his field."
    )
    author: str = "Unknown"


response = generate_object("Tell me a joke", response_model=Joke)

print(response)

In [ ]:
import ollama

from pydantic import BaseModel


class Joke(BaseModel):
    joke: str
    author: str


prompt = f"""
"Tell me a joke. 
Respond with the schema: {Joke.model_json_schema()}"
Include only the keys and values and not the schema itself.
"""

response = generate_object(prompt, response_model=Joke)

print(response)

In [ ]:
import ollama

from pydantic import BaseModel


class Joke(BaseModel):
    joke: str = "A joke"
    author: str = "An author"


def build_dummy_pydantic_object(schema: BaseModel) -> BaseModel:
    """
    Build a dummy Pydantic object using the given schema.

    Args:
      schema: The Pydantic schema to build the object from

    Returns:
      BaseModel: The dummy Pydantic object
    """
    return schema()


prompt = f"""
"Tell me a joke. 
Respond with the schema — Example object: {build_dummy_pydantic_object(Joke).model_dump_json()}"
Include only the keys and values and not the schema itself.
Respond strictly in the given schema format.
Respond only with valid JSON and don't include anything else.
"""

response = ollama.chat("phi4", messages=[{"role": "user", "content": prompt}])

print(response.message.content)